In [1]:
import pandas as pd
import seaborn as sb

#df = pd.read_csv('forestfires.csv')
df_portugal = pd.read_csv('data/Portugal_Dataset.csv')
df_portugal
#sb.heatmap(df.corr(), annot = True, fmt='.1g')


FileNotFoundError: ignored

In [ ]:
import pandas as pd
import seaborn as sb

df_cal_weather = pd.read_csv('data/California_Weather.csv' )

#df_cal_weather

In [ ]:
# Remove irrelevant columns
columns_to_keep_cal_weather = [
    'Stn Name',
    'CIMIS Region', 
    'Date', 
    'Precip (in)',
    #'Max Air Temp (F)',
    #'Min Air Temp (F)',
    'Avg Air Temp (F)',
    #'Max Rel Hum (%)',
    #'Min Rel Hum (%)',
    'Avg Rel Hum (%)',
    'Avg Wind Speed (mph)',
    'Wind Run (miles)'
]

df_cal_weather = df_cal_weather[columns_to_keep_cal_weather]
df_cal_weather.shape

(128125, 8)

In [ ]:
# Removing all missing data
df_cal_weather = df_cal_weather.dropna()
print(df_cal_weather.shape)

(128125, 8)
(128112, 8)


In [ ]:
# Show updated dataset
df_cal_weather
df_cal_weather.to_csv('data/California_Weather_clean.csv')

,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles)
0,FivePoints,San Joaquin Valley,1/1/2018,0.00,47.8,65.0,3.3,78.3
1,FivePoints,San Joaquin Valley,1/2/2018,0.00,47.2,67.0,3.1,74.5
2,FivePoints,San Joaquin Valley,1/3/2018,0.00,49.9,68.0,4.5,107.5
3,FivePoints,San Joaquin Valley,1/4/2018,0.01,56.8,74.0,5.8,140.2
4,FivePoints,San Joaquin Valley,1/5/2018,0.00,59.8,72.0,4.2,101.4
...,...,...,...,...,...,...,...,...
128120,Linden,San Joaquin Valley,9/14/2020,0.00,66.3,70.0,3.4,81.7
128121,Linden,San Joaquin Valley,9/15/2020,0.00,66.0,69.0,2.7,64.2
128122,Linden,San Joaquin Valley,9/16/2020,0.00,68.0,69.0,3.3,78.2
128123,Linden,San Joaquin Valley,9/17/2020,0.00,70.4,65.0,3.3,78.1


In [ ]:
import pandas as pd
import seaborn as sb

df_cal_fire = pd.read_csv('data/California_Fire_Incidents.csv')
#df_cal_fire


In [ ]:
# Remove irrelevant columns
columns_to_keep_cal_fire = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'Counties', 
    'Extinguished',
    'Location', 
    'MajorIncident', 
    'Started'
]

df_cal_fire = df_cal_fire[columns_to_keep_cal_fire]
df_cal_fire.shape

(1636, 8)

In [ ]:
# Removing all missing data
df_cal_fire = df_cal_fire.dropna()
print(df_cal_fire.shape)

(1636, 8)
(1577, 8)


In [ ]:
# Show updated dataset
df_cal_fire
df_cal_fire.to_csv('data/California_Fire_Incidents_clean.csv')

,AcresBurned,ArchiveYear,CalFireIncident,Counties,Extinguished,Location,MajorIncident,Started
0,257314.0,2013,True,Tuolumne,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,False,2013-08-17T15:25:00Z
1,30274.0,2013,True,Los Angeles,2013-06-08T18:30:00Z,Angeles National Forest,False,2013-05-30T15:28:00Z
2,27531.0,2013,True,Riverside,2013-07-30T18:00:00Z,Hwy 243 & Hwy 74 near Mountain Center,False,2013-07-15T13:43:00Z
3,27440.0,2013,False,Placer,2013-08-30T08:00:00Z,"Deadwood Ridge, northeast of Foresthill",False,2013-08-10T16:30:00Z
4,24251.0,2013,True,Ventura,2013-05-11T06:30:00Z,Southbound Highway 101 at Camarillo Springs Ro...,True,2013-05-02T07:01:00Z
...,...,...,...,...,...,...,...,...
1627,10.0,2019,True,Riverside,2019-09-10T10:45:00Z,Bailey Road and Terwilliger Road in Anza,False,2019-09-10T10:43:58Z
1629,10.0,2019,True,Siskiyou,2019-06-21T14:23:00Z,"Deer Run Road, east of Iron Gate Reservoir",False,2019-06-16T20:33:00Z
1630,10.0,2019,True,Butte,2019-05-08T08:47:00Z,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",False,2019-04-30T12:20:00Z
1631,9.0,2019,True,Riverside,2019-10-10T18:11:00Z,"Eagle Canyon Rd. and Cajalco Rd., southwest of...",False,2019-10-10T12:08:00Z
